<a href="https://colab.research.google.com/github/p3bozuric/headshot_generator/blob/main/headshot_generator_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is no-code tuning for headshot generator AI model.

This notebook is build to be run in Google Colab.

A100 GPU would be optimal for running this training. Depending on settings you set it will take a couple of hours. Make sure you keep this session running.

## Giving access to Google Drive

Manual approval to Google is mandatory after running next cell.




In [ ]:
from google.colab import drive
from google.colab import userdata

drive.mount('/content/drive')
hf_token = userdata.get('HF_TOKEN')

## Preparing AI toolkit environment

In [ ]:
!git clone https://github.com/ostris/ai-toolkit
!cd ai-toolkit && git submodule update --init --recursive && pip install -r requirements.txt

Cloning into 'ai-toolkit'...
remote: Enumerating objects: 3769, done.
remote: Counting objects: 100% (2323/2323), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 3769 (delta 2216), reused 2144 (delta 2129), pack-reused 1446 (from 1)
Receiving objects: 100% (3769/3769), 29.64 MiB | 35.13 MiB/s, done.
Resolving deltas: 100% (2870/2870), done.


## Huggingface token preparation (HF_TOKEN)

How to get & prepare HF_TOKEN:
1. Log in to huggingface
2. Create a token here: https://huggingface.co/settings/tokens
3. You need to click on the key icon to the left and place your token there under the name 'HF_TOKEN'.

In [ ]:
import os

# Set the environment variable
os.environ['HF_TOKEN'] = hf_token

print("HF_TOKEN environment variable has been set.")

HF_TOKEN environment variable has been set.


## Importing packages


In [ ]:
import os
import sys
sys.path.append('/content/ai-toolkit')
from toolkit.job import run_job
from collections import OrderedDict
from PIL import Image
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

## Dataset preparation

1. Dataset needs to be in folder **flux_dataset** which is inside of output directory you set in next cell.

2. Dataset needs to have 20-30 images of your face from different angles and backgrounds in various situations and face expressions.

3. Images should have names like: image001.jpg

4. Corresponding .txt files explaining the content of those images should be named like so: image001.txt

Keep in mind that .txt files are not necessary, but fine tuning will be better with written content of images.

# Hyperparameter setup

Fill out the form and run the cell.


In [ ]:
#@markdown ---
#@markdown ## **Project Configuration**
#@markdown Title of your project
project_name = 'professional_headshot_generator' # @param {type:"string"}
#@markdown Model you'll be fine-tuning. This code is optimized for FLUX.1-dev.
model_name = 'black-forest-labs/FLUX.1-dev' # @param ["black-forest-labs/FLUX.1-dev"]
#@markdown Where you want your model to be saved at. Keep in mind "/content/drive/MyDrive/" should be a constant.
output_dir = '/content/drive/MyDrive/headshot-generator' # @param {type:"string"}
#@markdown Instance prompt is a unique code by which you refer to yourself when prompting the generator in the inference.
instance_prompt = "pa3k" # @param {type:"string"}

#@markdown ---
#@markdown ## **Training Configuration**
#@markdown These parameters control the training process of your model.

#@markdown Number of images processed in one iteration.
batch_size = 2 # @param {type:"integer"}

#@markdown Total number of training iterations. 1000-4000 is a good range.
total_steps = 2000 # @param {type:"integer"}

#@markdown Rate at which the model learns. Higher values may lead to faster learning but potential instability.
learning_rate = 1e-4 # @param {type:"number"}

#@markdown Image resolutions to use during training. FLUX model benefits from multiple resolutions.
resolution = [512, 768, 1024] # @param {type:"raw"}

#@markdown Whether to train the U-Net part of the model. Usually kept True.
train_unet = True # @param {type:"boolean"}

#@markdown Whether to train the text encoder. Usually False for FLUX models.
train_text_encoder = False # @param {type:"boolean"}

#@markdown Use 8-bit Adam optimizer for reduced memory usage. Recommended if your GPU supports it.
use_8bit_adam = True # @param {type:"boolean"}

#@markdown Saves memory by doing forward/backward passes in chunks. Needed unless you have a lot of VRAM.
use_gradient_checkpointing = True # @param {type:"boolean"}

#@markdown Use Exponential Moving Average for more stable training. Recommended to leave on.
use_ema = True # @param {type:"boolean"}

#@markdown EMA decay rate. Higher values give more weight to recent iterations. Best to leave at 0.99.
ema_decay = 0.99 # @param {type:"number"}

#@markdown Use bfloat16 precision. Speeds up training if your GPU supports it.
use_bf16 = True # @param {type:"boolean"}

#@markdown ---
#@markdown ## **Sampling Configuration**
#@markdown These settings control the generation of test images during the training process.

#@markdown Generate sample images every this many steps.
sample_every = 500 # @param {type:"integer"}

#@markdown Width of the generated sample images.
sample_width = 1024 # @param {type:"integer"}

#@markdown Height of the generated sample images.
sample_height = 1024 # @param {type:"integer"}

#@markdown How closely the image adheres to the prompt. Higher values = closer adherence.
guidance_scale = 4 # @param {type:"number"}

#@markdown Number of denoising steps in image generation. More steps = potentially higher quality but slower.
sample_steps = 40 # @param {type:"integer"}

from collections import OrderedDict

job_to_run = OrderedDict([
    ('job', 'extension'),
    ('config', OrderedDict([
        # this name will be the folder and filename name
        ('name', project_name),
        ('process', [
            OrderedDict([
                ('type', 'sd_trainer'),
                # root folder to save training sessions/samples/weights
                ('training_folder', output_dir),
                # uncomment to see performance stats in the terminal every N steps
                ('performance_log_every', 500),
                ('device', 'cuda:0'),
                # if a trigger word is specified, it will be added to captions of training data if it does not already exist
                # alternatively, in your captions you can add [trigger] and it will be replaced with the trigger word
                ('trigger_word', instance_prompt),
                ('network', OrderedDict([
                    ('type', 'lora'),
                    ('linear', 16),
                    ('linear_alpha', 16)
                ])),
                ('save', OrderedDict([
                    ('dtype', 'float16'),  # precision to save
                    ('save_every', 500),  # save every this many steps
                    ('max_step_saves_to_keep', 4)  # how many intermittent saves to keep
                ])),
                ('datasets', [
                    # datasets are a folder of images. captions need to be txt files with the same name as the image
                    # for instance image2.jpg and image2.txt. Only jpg, jpeg, and png are supported currently
                    # images will automatically be resized and bucketed into the resolution specified
                    OrderedDict([
                        ('folder_path', f'{output_dir}/flux_dataset'),
                        ('caption_ext', 'txt'),
                        ('caption_dropout_rate', 0.05),  # will drop out the caption 5% of time
                        ('shuffle_tokens', False),  # shuffle caption order, split by commas
                        ('cache_latents_to_disk', True),  # leave this true unless you know what you're doing
                        ('resolution', resolution)  # flux enjoys multiple resolutions
                    ])
                ]),
                ('train', OrderedDict([
                    ('batch_size', batch_size),
                    ('steps', total_steps),  # total number of steps to train 500 - 4000 is a good range
                    ('gradient_accumulation_steps', 2),
                    ('train_unet', train_unet),
                    ('train_text_encoder', train_text_encoder),  # probably won't work with flux
                    ('content_or_style', 'content'),  # content, style, balanced
                    ('gradient_checkpointing', use_gradient_checkpointing),  # need this on unless you have a ton of vram
                    ('noise_scheduler', 'flowmatch'),  # for training only
                    ('optimizer', 'adamw8bit' if use_8bit_adam else 'adamw'),
                    ('lr', learning_rate),
                    # uncomment this to skip the pre training sample
                    ('skip_first_sample', True),

                    # ema will smooth out learning, but could slow it down. Recommended to leave on.
                    ('ema_config', OrderedDict([
                        ('use_ema', use_ema),
                        ('ema_decay', ema_decay)
                    ])),

                    # will probably need this if gpu supports it for flux, other dtypes may not work correctly
                    ('dtype', 'bf16' if use_bf16 else 'float32')
                ])),
                ('model', OrderedDict([
                    # huggingface model name or path
                    ('name_or_path', model_name),
                    ('is_flux', True),
                    ('quantize', True),  # run 8bit mixed precision
                    #('low_vram', True),  # uncomment this if the GPU is connected to your monitors. It will use less vram to quantize, but is slower.
                ])),
                ('sample', OrderedDict([
                    ('sampler', 'flowmatch'),  # must match train.noise_scheduler
                    ('sample_every', sample_every),  # sample every this many steps
                    ('width', sample_width),
                    ('height', sample_height),
                    ('prompts', [
                        # you can add [trigger] to the prompts here and it will be replaced with the trigger word
                        f'professional headshot of {instance_prompt} in a suit, studio lighting, neutral background',
                        f'business portrait of {instance_prompt} smiling, office setting, soft lighting',
                        f'corporate headshot of {instance_prompt} with confident expression, blurred office background',
                        f'professional profile picture of {instance_prompt} in business casual attire, outdoors',
                        f'LinkedIn profile photo of {instance_prompt} with friendly expression, solid color background',
                        f'{instance_prompt} giving a presentation in a conference room, professional attire',
                        f'close-up portrait of {instance_prompt} for company website, modern office background',
                        f'{instance_prompt} in a casual business meeting, gesturing while speaking, natural light'
                    ]),
                    ('neg', ''),  # not used on flux
                    ('seed', 42),
                    ('walk_seed', True),
                    ('guidance_scale', guidance_scale),
                    ('sample_steps', sample_steps)
                ]))
            ])
        ])
    ])),
    # you can add any additional meta info here. [name] is replaced with config name at top
    ('meta', OrderedDict([
        ('name', project_name),
        ('version', '1.0')
    ]))
])

# Start the training with cell bellow when you're ready

This might take a while. Keep the session running while training.

In [ ]:
run_job(job_to_run)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/local/lib/python3.10/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/usr/local/lib/python3.10

{
    "type": "sd_trainer",
    "training_folder": "/content/drive/MyDrive/headshot-generator",
    "performance_log_every": 500,
    "device": "cuda:0",
    "trigger_word": "pa3k",
    "network": {
        "type": "lora",
        "linear": 16,
        "linear_alpha": 16
    },
    "save": {
        "dtype": "float16",
        "save_every": 500,
        "max_step_saves_to_keep": 4
    },
    "datasets": [
        {
            "folder_path": "/content/drive/MyDrive/headshot-generator/flux_dataset_v2",
            "caption_ext": "txt",
            "caption_dropout_rate": 0.05,
            "shuffle_tokens": false,
            "cache_latents_to_disk": true,
            "resolution": [
                512,
                768,
                1024
            ]
        }
    ],
    "train": {
        "batch_size": 2,
        "steps": 3000,
        "gradient_accumulation_steps": 2,
        "train_unet": true,
        "train_text_encoder": false,
        "content_or_style": "content",
      

transformer/config.json:   0%|          | 0.00/378 [00:00<?, ?B/s]

...

Saving at step 2500


pa3k_professional_headshot_generator_v2:  83%|████████▎ | 2499/3000 [2:05:10<23:53,  2.86s/it, lr: 1.0e-04 loss: 2.919e-01]

Saved to /content/drive/MyDrive/headshot-generator/pa3k_professional_headshot_generator_v2/optimizer.pt
Removing old save: /content/drive/MyDrive/headshot-generator/pa3k_professional_headshot_generator_v2/pa3k_professional_headshot_generator_v2_000000500.safetensors

Timer 'pa3k_professional_headshot_generator_v2 Timer':
 - 2.8598s avg - train_loop, num = 10
 - 1.7714s avg - backward, num = 10
 - 0.8108s avg - predict_unet, num = 10
 - 0.2687s avg - reset_batch, num = 6
 - 0.1208s avg - optimizer_step, num = 10
 - 0.1088s avg - calculate_loss, num = 10
 - 0.0687s avg - encode_prompt, num = 10
 - 0.0029s avg - get_batch, num = 10
 - 0.0028s avg - preprocess_batch, num = 10
 - 0.0014s avg - prepare_noise, num = 10
 - 0.0005s avg - batch_cleanup, num = 10
 - 0.0004s avg - prepare_latents, num = 10
 - 0.0001s avg - grad_setup, num = 10
 - 0.0000s avg - scheduler_step, num = 10
 - 0.0000s avg - prepare_prompt, num = 10
 - 0.0000s avg - log_to_tensorboard, num = 5



pa3k_professional_headshot_generator_v2: 100%|█████████▉| 2999/3000 [2:30:01<00:03,  3.00s/it, lr: 1.0e-04 loss: 4.236e-01]



Saved to /content/drive/MyDrive/headshot-generator/pa3k_professional_headshot_generator_v2/optimizer.pt
